In [1]:
#%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.tsa.stattools as ts
from init_objects import *
from functions.stylizedfacts import *
from functions.helpers import organise_data
from functions.evolutionaryalgo import quadratic_loss_function
import statsmodels.api as sm
from matplotlib import style
import json
from collections import OrderedDict
import seaborn as sns
import scipy.stats as stats
import scipy
import random

In [2]:
style.use('ggplot')

# Model dynamics

In [3]:
params = {"spread_max": 0.004087, "fundamental_value": 166, 
          "trader_sample_size": 19, "n_traders": 1000, 
          "ticks": 2500, "std_fundamental": 0.0530163128919286, 
          "std_noise": 0.10696588473846724, "w_mean_reversion": 93.63551013606137, 
          "w_fundamentalists": 8.489180919376432, "w_momentum": 43.055017297045524, 
          "max_order_expiration_ticks": 30, "std_vol": 7, "w_random": 73.28414619497076, 
          "horizon_max": 10}

## Change model to collect correct data

1. close_price, fundamental
2. agents: type, selected agents
3. bid_book orders, price, size

In [4]:
def sim_fin_model(traders, orderbook, parameters, seed=1):
    """
    The main model function
    :param traders: list of Agent objects
    :param orderbook: object Order book
    :param parameters: dictionary of parameters
    :param seed: integer seed to initialise the random number generators
    :return: list of simulated Agent objects, object simulated Order book
    """
    close_price = []
    fundamental_val = []
    volume = []
    
    random.seed(seed)
    np.random.seed(seed)
    fundamental = [parameters["fundamental_value"]]

    for tick in range(parameters['horizon_max'] + 1, parameters["ticks"]):
        # evolve the fundamental value via random walk process
        fundamental.append(fundamental[-1] + parameters["std_fundamental"] * np.random.randn())

        # select random sample of active traders
        active_traders = random.sample(traders, int((parameters['trader_sample_size'])))

        # update common expectation components
        mid_price = orderbook.tick_close_price[-1]
        fundamental_component = np.log(fundamental[-1] / mid_price)
        chartist_component = np.cumsum(orderbook.returns[:-parameters['horizon_max']-1:-1]
                                       ) / np.arange(1., float(parameters['horizon_max'] + 1))

        for trader in active_traders:
            # update trader specific expectations
            noise_component = parameters['std_noise'] * np.random.randn()

            fcast_return = trader.var.forecast_adjust * (
                trader.var.weight_fundamentalist * fundamental_component +
                trader.var.weight_chartist * chartist_component[trader.par.horizon] +
                trader.var.weight_random * noise_component -
                trader.var.weight_mean_reversion * chartist_component[trader.par.horizon])

            fcast_price = mid_price * np.exp(fcast_return)

            # submit orders
            if fcast_price > mid_price:
                bid_price = fcast_price * (1. - trader.par.spread)
                orderbook.add_bid(bid_price, abs(int(np.random.normal(scale=parameters['std_vol']))), trader)
            elif fcast_price < mid_price:
                ask_price = fcast_price * (1 + trader.par.spread)
                orderbook.add_ask(ask_price, abs(int(np.random.normal(scale=parameters['std_vol']))), trader)

        # match orders in the order-book
        while True:
            matched_orders = orderbook.match_orders()
            if matched_orders is None:
                break

        # clear and update order-book history
        volume.append(sum(orderbook.transaction_volumes))
        orderbook.cleanse_book()
        fundamental_val.append(fundamental[-1])
        orderbook.fundamental = fundamental
        close_price.append(orderbook.tick_close_price[-1])
        

    return traders, orderbook, close_price, fundamental_val, volume


## Run model

In [5]:
NRUNS = 1
traders = []
obs = []
for seed in range(NRUNS): 
    traders, orderbook = init_objects(params, seed)
    traders, orderbook, close_price, fundamental_val, volume = sim_fin_model(traders, orderbook, params, seed)
    traders.append(traders)
    obs.append(orderbook)

In [6]:
#close_price 
#fundamental_val
#volume

## Create graph of prices, fundamental, vol

In [7]:
%matplotlib tk
from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation

In [8]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12,9))

ax1.plot(range(len(close_price)), fundamental_val, label='fundamental value')
ax1.plot(range(len(close_price)), close_price, label='close price')

ax1.set_ylabel('End tick price', fontsize='14')
ax1.legend(loc='best', fontsize='14')

ax2.bar(range(len(volume)), volume, width=1.0)
ax2.set_ylabel('Volume', fontsize='14')
ax2.set_xlabel('Ticks', fontsize='14')

# for ax in [ax1, ax2]:
#     ax.ticklabel_format(axis='y', style='sci', scilimits=(-2,2))

Text(0.5,0,'Ticks')

## Animate

In [9]:
animation.writers.avail

{u'html': matplotlib.animation.HTMLWriter,
 u'pillow': matplotlib.animation.PillowWriter}

In [10]:
# Set up formatting for the movie files
Writer = animation.writers['pillow']
writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)

In [51]:
animation_time = 1000

def data_gen(t=0):
    cnt = 0
    while t < animation_time:
        cnt += 1
        t += 1
        yield t, close_price[cnt], fundamental_val[cnt], volume[cnt]


def init():
    ax.set_ylim(150, 180)
    ax.set_xlim(0, animation_time)
    
    ax2.set_ylim(0, 180)
    ax2.set_xlim(0, animation_time)
    
    del xdata[:]
    del ydata[:]
    del fdata[:]
    del vol_data[:]
    line.set_data(xdata, ydata)
    line2.set_data(xdata, fdata)
    line3.set_data(xdata, vol_data)
    ax.legend((line, line2), ('close_price', 'fundamental'), fontsize='12')
    ax2.legend()
    return line,

fig, (ax, ax2) = plt.subplots(2, 1)
line, = ax.plot([], [], lw=2)
line2, = ax.plot([], [], lw=2)
# add vol
line3, = ax2.plot([], [], lw=2, color=list(plt.rcParams['axes.prop_cycle'])[4]['color'], label='volume', marker="|", linestyle="None")#, lw=2, marker=".", linestyle="None")



ax2.plot()

ax.grid()
xdata, ydata, fdata, vol_data = [], [], [], []


def run(data):
    # update the data
    t, y, fun, vol = data
    xdata.append(t)
    ydata.append(y)
    fdata.append(fun)
    vol_data.append(vol)
    line.set_data(xdata, ydata)
    line2.set_data(xdata, fdata)
    line3.set_data(xdata, vol_data)

    return line,

ani = animation.FuncAnimation(fig, run, data_gen, blit=False, interval=1,
                              repeat=False, init_func=init, save_count=animation_time)

ani.save('price_fundamental_vol.gif', writer=writer)
plt.show()

## Agents

1. Show all agents as grey dots around the screen edges. 
2. Show active agents in middle
3. Show buying agents green, show selling agents red

data needed:
np of agents,


In [104]:
class AnimatedScatter(object):
    """An animated scatter plot using matplotlib.animations.FuncAnimation."""
    def __init__(self, numpoints=50):
        self.numpoints = numpoints
        self.stream = self.data_stream()

        # Setup the figure and axes...
        self.fig, self.ax = plt.subplots()
        # Then setup FuncAnimation.
        self.ani = animation.FuncAnimation(self.fig, self.update, interval=5, 
                                           init_func=self.setup_plot, blit=True)

    def setup_plot(self):
        """Initial drawing of the scatter plot."""
        x, y, s, c = next(self.stream)
        self.scat = self.ax.scatter(x, y, c=c, s=s, animated=True)
        self.ax.axis([-10, 10, -10, 10])

        # For FuncAnimation's sake, we need to return the artist we'll be using
        # Note that it expects a sequence of artists, thus the trailing comma.
        return self.scat,

    def data_stream(self):
        """Generate a random walk (brownian motion). Data is scaled to produce
        a soft "flickering" effect."""
        data = np.random.random((4, self.numpoints))
        xy = data[:2, :]
        s, c = data[2:, :]
        xy -= 0.5
        xy *= 10
        while True:
            xy += 0.03 * (np.random.random((2, self.numpoints)) - 0.5)
            s += 0.05 * (np.random.random(self.numpoints) - 0.5)
            c += 0.02 * (np.random.random(self.numpoints) - 0.5)
            yield data

    def update(self, i):
        """Update the scatter plot."""
        data = next(self.stream)

        # Set x and y data...
        self.scat.set_offsets(data[:2, :].reshape(self.numpoints, 2))
        # Set sizes...
        self.scat._sizes = 300 * abs(data[2])**1.5 + 100
        # Set colors..
        self.scat.set_array(data[3])

        # We need to return the updated artist for FuncAnimation to draw..
        # Note that it expects a sequence of artists, thus the trailing comma.
        return self.scat,

    def show(self):
        plt.show()

if __name__ == '__main__':
    a = AnimatedScatter()
    a.show()

Traceback (most recent call last):
  File "C:\Users\joerischasfoort\Anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 387, in process
    proxy(*args, **kwargs)
  File "C:\Users\joerischasfoort\Anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 227, in __call__
    return mtd(*args, **kwargs)
  File "C:\Users\joerischasfoort\Anaconda3\lib\site-packages\matplotlib\animation.py", line 1308, in _handle_resize
    self._init_draw()
  File "C:\Users\joerischasfoort\Anaconda3\lib\site-packages\matplotlib\animation.py", line 1753, in _init_draw
    self._drawn_artists = self._init_func()
  File "<ipython-input-104-1d10c0b39103>", line 16, in setup_plot
    self.scat = self.ax.scatter(x, y, c=c, s=s, animated=True)
  File "C:\Users\joerischasfoort\Anaconda3\lib\site-packages\matplotlib\__init__.py", line 1867, in inner
    return func(ax, *args, **kwargs)
  File "C:\Users\joerischasfoort\Anaconda3\lib\site-packages\matplotlib\axes\_axes.py", line 4334, in scatter
   

In [66]:
# animation steps:
# setup plotting area
fig, ax = plt.subplots(figsize=(5, 3))
ax.set(xlim=(-3, 3), ylim=(-1, 1))

# create data to plot
x = np.linspace(-3, 3, 91)
t = np.linspace(1, 25, 30)
X2, T2 = np.meshgrid(x, t)
 
sinT2 = np.sin(2*np.pi*T2/T2.max())
F = 0.9*sinT2*np.sinc(X2*(1 + sinT2))

# setup line to plot 
line = ax.plot(x, F[0, :], color='k', lw=2)[0]

# create function to update line
def animate(i):
    line.set_ydata(F[i, :])

# create animation
anim = FuncAnimation(fig, animate, interval=100, frames=len(t)-1, save_count=animation_time)

# optional save graph. ani.save('price_fundamental_vol.gif', writer=writer)

plt.draw()
plt.show()

In [115]:
# setup plotting area
fig, ax = plt.subplots(figsize=(5, 3))
ax.set(xlim=(-3, 3), ylim=(-1, 1))

# create data
x = np.linspace(-3, 3, 91)
t = np.linspace(1, 25, 30)
X2, T2 = np.meshgrid(x, t)
 
sinT2 = np.sin(2*np.pi*T2/T2.max())
F = 0.9*sinT2*np.sinc(X2*(1 + sinT2))

# create scatter objects
scat = ax.scatter(x[::3], F[0, ::3])

# define animation function
def animate(i):
    y_i = F[i, ::3]
    scat.set_offsets(np.c_[x[::3], y_i])
    
anim = FuncAnimation(fig, animate, interval=100, frames=len(t)-1, save_count=animation_time)

# optional save graph. ani.save('price_fundamental_vol.gif', writer=writer)

plt.draw()
plt.show()

# Scatter plot agents

In [112]:
F

array([[ -1.36130895e-02,  -1.69609663e-02,  -1.92842798e-02, ...,
         -1.92842798e-02,  -1.69609663e-02,  -1.36130895e-02],
       [  2.52482749e-02,   2.01056101e-02,   1.28580299e-02, ...,
          1.28580299e-02,   2.01056101e-02,   2.52482749e-02],
       [  1.58908956e-02,   2.65063688e-02,   3.45273017e-02, ...,
          3.45273017e-02,   2.65063688e-02,   1.58908956e-02],
       ..., 
       [  4.00560568e-02,   4.71254578e-02,   5.37735218e-02, ...,
          5.37735218e-02,   4.71254578e-02,   4.00560568e-02],
       [ -2.31201752e-02,  -2.17782898e-02,  -1.97599170e-02, ...,
         -1.97599170e-02,  -2.17782898e-02,  -2.31201752e-02],
       [ -9.16875209e-32,  -4.97337237e-18,  -9.95564925e-18, ...,
         -9.95564925e-18,  -4.97337237e-18,  -9.16875209e-32]])

In [141]:
list(plt.rcParams['axes.prop_cycle'])[4]['color']

'#FBC15E'

In [166]:
# setup plotting area
fig, ax = plt.subplots(figsize=(10, 8))
ax.set(xlim=(-3, 3), ylim=(-1, 1))

offsets = np.linspace(-3, 3, 91)

# todo add color

# agents = list of

# create scatter objects
scat = ax.scatter([], [], color=list(plt.rcParams['axes.prop_cycle'])[3]['color'])

# define animation function
def animate(frame_number):
    np.random.seed(0)
    x = list(np.random.randint(-300, 100, 1000-params['trader_sample_size']) / 100.0)
    y = np.random.randint(-100, 100, 1000) / 100.0
    np.random.seed(None)
    x = np.array(x + list(np.random.randint(100, 300, params['trader_sample_size']) / 100.0))
    
    x = x + random.gauss(0, 0.01)
    y = y + random.gauss(0, 0.01)
    # set position
    scat.set_offsets(zip(x, y))
    
    color = np.where(x>1,'g',np.where(y>5,'k','k'))
    # give red color to some sellers
    color[-params['trader_sample_size']:] = np.random.choice(['g', 'r'], size=params['trader_sample_size'], replace=True)
    #print(color)
    # set random number of 
    
    # set color
    scat.set_color(color)
    
anim = FuncAnimation(fig, animate, interval=100, frames=animation_time, save_count=animation_time)

anim.save('agents.gif', writer=writer)

plt.draw()
plt.show()